In [1]:
from main import *

In [2]:

# YEARS = [2018, 2019, 2021, 2022, 2023, 2024]
# SAVE_DIR = "data"
#
# fetch_lap_data(YEARS, save_dir=SAVE_DIR)
# fetch_car_data(YEARS, save_dir=SAVE_DIR)



In [3]:
# Set parameters
YEARS = [2018, 2019, 2021, 2022, 2023, 2024]
DATA_DIR = "data"

# Step 1: Annotate telemetry data with LapNumber for each year
for year in YEARS:
    annotate_lap_number(year, data_dir=DATA_DIR)

# Step 2: Aggregate telemetry per lap and per driver
df_all_years = aggregate_car_data(YEARS, data_dir=DATA_DIR)

# Step 3: Display preview
df_all_years.head()


Annotating lap numbers for 2018...
Saved annotated car data to data/car_data_with_lap_2018.csv with 505054 rows.
Annotating lap numbers for 2019...
Saved annotated car data to data/car_data_with_lap_2019.csv with 505445 rows.
Annotating lap numbers for 2021...
Saved annotated car data to data/car_data_with_lap_2021.csv with 455972 rows.
Annotating lap numbers for 2022...
Saved annotated car data to data/car_data_with_lap_2022.csv with 487790 rows.
Annotating lap numbers for 2023...
Saved annotated car data to data/car_data_with_lap_2023.csv with 486247 rows.
Annotating lap numbers for 2024...
Saved annotated car data to data/car_data_with_lap_2024.csv with 518492 rows.
Aggregating car data for 2018...
Saved: data/aggregated_car_data_2018.csv
Aggregating car data for 2019...
Saved: data/aggregated_car_data_2019.csv
Aggregating car data for 2021...
Saved: data/aggregated_car_data_2021.csv
Aggregating car data for 2022...
Saved: data/aggregated_car_data_2022.csv
Aggregating car data for 2

,DriverNumber,LapNumber,RPM,nGear,Throttle,Brake,DRS,Year
0,2,1,9241.715976,3.328402,40.363905,105,0,2018
1,2,2,9468.847305,3.694611,43.904192,98,0,2018
2,2,3,9465.868657,3.785075,44.597015,107,7,2018
3,2,4,9417.035821,3.817910,42.958209,101,19,2018
4,2,5,9434.817365,3.850299,42.473054,102,19,2018


In [4]:
df_all_years

,DriverNumber,LapNumber,RPM,nGear,Throttle,Brake,DRS,Year
0,2,1,9241.715976,3.328402,40.363905,105,0,2018
1,2,2,9468.847305,3.694611,43.904192,98,0,2018
2,2,3,9465.868657,3.785075,44.597015,107,7,2018
3,2,4,9417.035821,3.817910,42.958209,101,19,2018
4,2,5,9434.817365,3.850299,42.473054,102,19,2018
...,...,...,...,...,...,...,...,...
8306,81,74,9361.694444,3.638889,44.204861,78,0,2024
8307,81,75,9338.838596,3.603509,45.568421,79,0,2024
8308,81,76,9425.898955,3.675958,44.073171,80,0,2024
8309,81,77,9407.110345,3.586207,45.710345,80,0,2024
